In [81]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

from pymilvus import MilvusClient
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
from IPython.display import display, Image, HTML

### Configurar Milvus - Base de datos de Vectores

In [ ]:
client = MilvusClient("fashion_demo.db")
if client.has_collection("fashion_demo"):
    print('exists')
client.drop_collection(collection_name="fashion_demo.db")

In [ ]:
DIMENSION = 128
COLLECTION_NAME = "fashion_recommender"

db_client = MilvusClient(uri="fashion.db")

def crear_coleccion():

    if db_client.has_collection(COLLECTION_NAME):
       db_client.drop_collection(COLLECTION_NAME)

    schema = MilvusClient.create_schema(
        auto_id=True,
        enable_dynamic_field=False
    )

    db_client.create_collection(
        collection_name=COLLECTION_NAME,
        vector_field_name="image_embeddings",
        dimension=DIMENSION,
        auto_id=True,
        enable_dynamic_field=True,
        metric_type="COSINE",
    )

def extraer_features(folder_imagen: str, nombre_imagen: str):
    # Extraer características de imágenes con SIFT
    sift = cv2.SIFT_create()
    ruta_imagen = os.path.join(folder_imagen, nombre_imagen)
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        print(f"No se pudo leer la imagen '{ruta_imagen}'.")
        return None
    keypoints, descriptors = sift.detectAndCompute(imagen, None)
    promedio_descriptores = np.mean(descriptors, axis=0)
    return promedio_descriptores.tolist()

def insertar_en_coleccion(folder_imagen: str):
    # Recorrer las imágenes en el directorio
    count = 0
    for nombre_imagen in os.listdir(folder_imagen):
        features_imagen = extraer_features(folder_imagen, nombre_imagen)
        if not features_imagen:
            print(f"No se pudo leer la imagen '{nombre_imagen}'.")
            continue

        db_client.insert(COLLECTION_NAME, {"image_id": nombre_imagen ,"image_embeddings": features_imagen })
        count += 1

        if count == 1000:
            break
    
    print(f"Total imagenes insertadas: {count}")


In [54]:
crear_coleccion()
folder_imagenes = "fashion-dataset/images"
insertar_en_coleccion(folder_imagenes)

Total imagenes insertadas: 1000


In [84]:
query_folder = "fashion-dataset/test_images"
query_image_name = "1590.jpg"
features = extraer_features(query_folder, query_image_name)

results = db_client.search(
    COLLECTION_NAME,
    data=[features],
    output_fields=["image_id"],
    search_params={"metric_type": "COSINE"}
)
results

data: ["[{'id': 453783406382155306, 'distance': 0.9960609674453735, 'entity': {'image_id': '8997.jpg'}}, {'id': 453783417167284094, 'distance': 0.9952723979949951, 'entity': {'image_id': '26526.jpg'}}, {'id': 453783375998616142, 'distance': 0.9951582551002502, 'entity': {'image_id': '28641.jpg'}}, {'id': 453783390461363244, 'distance': 0.9948838949203491, 'entity': {'image_id': '47053.jpg'}}, {'id': 453783410780669632, 'distance': 0.993597686290741, 'entity': {'image_id': '54629.jpg'}}, {'id': 453783394558936236, 'distance': 0.9935033917427063, 'entity': {'image_id': '6819.jpg'}}, {'id': 453783364616323310, 'distance': 0.9934462308883667, 'entity': {'image_id': '2882.jpg'}}, {'id': 453783374894465576, 'distance': 0.9929875731468201, 'entity': {'image_id': '20235.jpg'}}, {'id': 453783369736257946, 'distance': 0.9923688173294067, 'entity': {'image_id': '39943.jpg'}}, {'id': 453783372310512100, 'distance': 0.9921301007270813, 'entity': {'image_id': '11942.jpg'}}]"] 

In [85]:
for result in results:
    for hit in result[:10]:
        print(hit["entity"])

{'image_id': '8997.jpg'}
{'image_id': '26526.jpg'}
{'image_id': '28641.jpg'}
{'image_id': '47053.jpg'}
{'image_id': '54629.jpg'}
{'image_id': '6819.jpg'}
{'image_id': '2882.jpg'}
{'image_id': '20235.jpg'}
{'image_id': '39943.jpg'}
{'image_id': '11942.jpg'}


In [87]:
def display_products(results: list):
    html = ""
    for result in results:
        for hit in result[:10]:
            html += f"""
                <div style="display: inline-block; text-align: center; margin: 10px;">
                    <img src="fashion-dataset/images/{hit["entity"]["image_id"]}" width="150" /><br>
                    <b style="width=180">{hit["entity"]["image_id"]}</b><br>
                </div>
            """

    display(HTML(html))

display_products(results)